In [ ]:
import numpy as np
import pandas as pd
import tensorflow as tf

#Keras
from keras.models import Model, Sequential
from keras import utils
from keras.preprocessing import image

#Keras NLP
from keras.preprocessing.text import Tokenizer, text_to_word_sequence 
from keras.preprocessing.sequence import pad_sequences

#Sklearn
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler, MinMaxScaler, LabelEncoder

#Сторонние либы
import time
import os
import re
import librosa
from PIL import Image

#Базовые слои
from keras.layers import Input, concatenate, Activation, Reshape, Flatten, Dense, LSTM, GRU

#Свертка
from keras.layers import Conv1D, Conv2D, Conv3D, MaxPooling1D, MaxPooling2D, MaxPooling3D
from keras.layers import Conv1DTranspose, Conv2DTranspose, Conv3DTranspose
from keras.layers import GlobalMaxPooling1D, GlobalMaxPooling2D, GlobalMaxPooling3D

#Регуляризация
from keras.layers import Dropout, BatchNormalization
from keras.layers import SpatialDropout1D, SpatialDropout2D, SpatialDropout3D

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
genres = 'blues classical country disco hiphop jazz metal pop reggae rock'.split()

In [ ]:
def get_futures(y, sr):
    chroma_stft = librosa.feature.chroma_stft(y=y, sr=sr)
    mfcc = librosa.feature.mfcc(y=y, sr=sr)
    centroid = np.mean(librosa.feature.spectral_centroid(y=y, sr=sr))
    rolloff = np.mean(librosa.feature.spectral_rolloff(y=y, sr=sr))
    bw = np.mean(librosa.feature.spectral_bandwidth(y=y, sr=sr))
    zcr = np.mean(librosa.feature.zero_crossing_rate(y=y))
    rms = np.mean(librosa.feature.rms(y=y))

    out = []
    out.append(centroid)
    out.append(rolloff)
    out.append(bw)
    out.append(zcr)
    out.append(rms)

    for elem in mfcc:
        out.append(np.mean(elem))
      
    for elem in chroma_stft:
      out.append(np.mean(elem))

    return out

In [ ]:
y_train = []
x_train = []
for i in range(len(genres)):
    g = genres[i]
    for filename in os.listdir(f'/content/drive/MyDrive/PythonCode/УИИ/8 (Audio)/genres/{g}'):
        songname = f'/content/drive/MyDrive/PythonCode/УИИ/8 (Audio)/genres/{g}/{filename}'
        y, sr = librosa.load(songname, mono=True, duration=30)
        feature = get_futures(y, sr)
        x_train.append(feature)
        y_train.append(tf.keras.utils.to_categorical(i, len(genres)))



In [ ]:
x_train = np.array(x_train)
y_train = np.array(y_train)

In [ ]:
x_train.shape
indexes = range(0,37)

In [ ]:
inp = Input(shape=(37,))
x = Dense(128, activation='elu')(inp)
x = Dense(256, activation='elu')(x)
x = Dense(128, activation='elu')(x)
x = Dense(256, activation='elu')(x)
x = Dense(128, activation='elu')(x)
x = Dense(256, activation='elu')(x)
x = Dense(128, activation='elu')(x)
x = Dense(256, activation='elu')(x)
out = Dense(len(genres), activation='softmax')(x)

model = Model(inp, out)
model.compile(loss='categorical_crossentropy', metrics=['accuracy'], optimizer='adam')

In [ ]:
history = model.fit(x_train[:, indexes],
                    y_train,
                    epochs=150,
                    batch_size=20,
                    validation_split=0.2)

Epoch 1/150
40/40 [==============================] - 1s 12ms/step - loss: 0.9639 - accuracy: 0.6225 - val_loss: 9.7501 - val_accuracy: 0.0300
Epoch 2/150
40/40 [==============================] - 0s 7ms/step - loss: 0.8855 - accuracy: 0.6137 - val_loss: 9.9433 - val_accuracy: 0.0100
Epoch 3/150
40/40 [==============================] - 0s 6ms/step - loss: 0.8735 - accuracy: 0.6400 - val_loss: 9.6757 - val_accuracy: 0.0100
Epoch 4/150
40/40 [==============================] - 0s 6ms/step - loss: 0.8815 - accuracy: 0.6288 - val_loss: 10.2054 - val_accuracy: 0.0050
Epoch 5/150
40/40 [==============================] - 0s 7ms/step - loss: 0.8713 - accuracy: 0.6363 - val_loss: 11.4516 - val_accuracy: 0.0100
Epoch 6/150
40/40 [==============================] - 0s 8ms/step - loss: 0.8445 - accuracy: 0.6525 - val_loss: 10.5811 - val_accuracy: 0.0050
Epoch 7/150
40/40 [==============================] - 0s 7ms/step - loss: 0.8289 - accuracy: 0.6488 - val_loss: 10.6280 - val_accuracy: 0.0050
Epoch 8/